# basics
## building dataset

In [19]:
from torch.utils.data import Dataset
from torch import Tensor
class MyDataset(Dataset):
    def __getitem__(self, index):
        return Tensor([index, 0.]), Tensor([3. * index + 4])
    def __len__(self):
        return 400
training_dataset = MyDataset()
print(len(training_dataset))
print(training_dataset[0])
print(training_dataset[1])

400
(tensor([0., 0.]), tensor([4.]))
(tensor([1., 0.]), tensor([7.]))


## building dataloader

In [20]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_dataset, batch_size=4)
for x, y in train_dataloader:
    print(x)
    print(y)
    break

tensor([[0., 0.],
        [1., 0.],
        [2., 0.],
        [3., 0.]])
tensor([[ 4.],
        [ 7.],
        [10.],
        [13.]])


## building model

In [21]:
from torch import nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = nn.Linear(2, 1)
    def forward(self, x):
        return self.model(x)
model = MyModel()
print(model)

MyModel(
  (model): Linear(in_features=2, out_features=1, bias=True)
)


## building optimizer

In [22]:
from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=1e-1)

## training

In [23]:
loss_fn = nn.MSELoss()
for epoch in range(100):
    for batch, (x, y) in enumerate(train_dataloader):
        loss = loss_fn(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 19:
        print(f'epoch: {epoch}, loss: {loss.item()}')
        for n, p in model.named_parameters():
            print(f'name, parameters: {n, p}')
        print()

epoch: 19, loss: 1.2293457984924316e-07
name, parameters: ('model.weight', Parameter containing:
tensor([[2.9999, 0.2765]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([4.0463], requires_grad=True))

epoch: 39, loss: 0.0001651458442211151
name, parameters: ('model.weight', Parameter containing:
tensor([[2.9947, 0.2765]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([6.1100], requires_grad=True))

epoch: 59, loss: 72.84194946289062
name, parameters: ('model.weight', Parameter containing:
tensor([[2.9516, 0.2765]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([5.8897], requires_grad=True))

epoch: 79, loss: 0.006389569491147995
name, parameters: ('model.weight', Parameter containing:
tensor([[3.0070, 0.2765]], requires_grad=True))
name, parameters: ('model.bias', Parameter containing:
tensor([1.1720], requires_grad=True))

epoch: 99, loss: 3.1487696170806885
name, parameters

## testing

In [24]:
print(model(Tensor([100, 0])))
print(model(Tensor([10000000, 0])))

tensor([304.3310], grad_fn=<AddBackward0>)
tensor([30110418.], grad_fn=<AddBackward0>)


可以看到，这里泛化到`10000000`就有了不小的误差

## building lightning model

In [25]:
from pytorch_lightning import LightningModule
class MyLitModel(LightningModule):
    def __init__(self):
        super(MyLitModel, self).__init__()
        self.model = nn.Linear(2, 1)
    def forward(self, x):
        return self.model(x)
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-1)
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        loss = loss_fn(self.model(x), y)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        loss = loss_fn(self.model(x), y)
        self.log('val_loss', loss)
model = MyLitModel()
print(model)

MyLitModel(
  (model): Linear(in_features=2, out_features=1, bias=True)
)


## building trainer

In [26]:
from pytorch_lightning import Trainer
trainer = Trainer(max_epochs=10)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## training using pytorch_lightning
这里训练和验证集是同一个

In [27]:
trainer.fit(model, train_dataloader, train_dataloader)
model.eval()
inputs = Tensor([[i, 2*i] for i in range(4)])
print(model(inputs))


  | Name  | Type   | Params
---------------------------------
0 | model | Linear | 3     
---------------------------------
3         Trainable params
0         Non-trainable params
3         Total params
0.000     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 200/200 [00:01<00:00, 126.80it/s, loss=0.000162, v_num=1]
